# Item 50: Annotate Class Attributes with `__set_name__`

One more useful feature enabled by metaclasses is the ability to modify or annotate properties after a class is defined but before the class is actually used. This approach is commonly used with *descriptors* to give them more introspection into how we're being used within their containing class.

In [1]:
# Say for example that we want to define a class that represents a row in a customer database and we'd
# like to have a corresponding property on the class for each column in the database table
class Field:
    def __init__(self, name):
        self.name = name
        self.internal_name = '_' + self.name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

In [2]:
# Defining the class representing a row requires suppyling the database table's column name for each 
# class attribute
class Customer:
    # Class attributes
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')

In [3]:
# Here, we can see how the Field descriptors modify the instance dictionary __dict__ as expected
cust = Customer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Euclid'
print(f'After: {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After: 'Euclid' {'_first_name': 'Euclid'}


The class definition seems redundant on both sides (`first_name = Field('first_name')`).

The problem is the order of operations in the customer class (`Field` constructor is called (right) and then assigned to the variable (left)). There's no way for a `Field` instance to know upfront which class attribute it will be assigned to.

In [4]:
# To eliminate said redundancy, we can use a metaclass
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

In [5]:
# All clases representing database rows should inherit from this class to ensure that they use the metaclass
class DatabaseRow(metaclass=Meta):
    pass

In [6]:
# With the metaclass, it no longer requires arguments to be passed to its constructor. Instead, its
# attributes are set by the Meta.__new__ method above
class Field:
    def __init__(self):
        # These will be assigned by the metaclass
        self.name = None
        self.internal_name = None
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

In [7]:
# By using the metaclass, the new DatabaseRow base class, and the new Field descriptor, the class
# definition for a database row no longer has the redundancy from before
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

In [8]:
# The behavior of the new class is identical to the behavior of the old one
cust = Customer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Euler'
print(f'After: {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After: 'Euler' {'_first_name': 'Euler'}


In [9]:
# Problem: if we forget to inherit from DatabaseRow we can't ise Field class properties and our code
# will break
class BrokenCustomer:
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

cust = BrokenCustomer()
cust.first_name = 'Mersenne'

TypeError: attribute name must be string, not 'NoneType'

In [10]:
# To solve the problem above we can use the __set_name__ special method for descriptors. Here, we avoid
# defining a metaclass entirely and move what the Meta.__new__ method from above was doing into __set_name__
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None

    def __set_name__(self, owner, name):
        # Called on class creation for each descriptor
        self.name = name
        self.internal_name = '_' + name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)


In [11]:
# Now, we can get the benefits of the Field descriptor without having to inherit from a specific parent
# class or having to use a metaclass
class FixedCustomer:
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()


cust = FixedCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Mersenne'
print(f'After: {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After: 'Mersenne' {'_first_name': 'Mersenne'}
